# Exploratory Data Analysis wiht Deedle

This notebook tries to investigate the EDA capabilities of Deedle in **Dotnet Interactive Notebooks**. The different tasks are mainly inspired by the R community's [ModernDive Book](https://moderndive.com/index.html).

## Load Nuget Dependencies

The main dependency for this task is [Deedle](https://fslab.org/Deedle/) - .NET OSS implementation of the data frame concept known from the R programming languages and `pandas`.
In .NET notebooks you can load external dependencies directly from nuget.

In [1]:
#!fsharp
#r "nuget: Deedle"
#r "nuget: XPlot.Plotly"

Installed package XPlot.Plotly version 3.0.1

Installed package Deedle version 2.3.0

## Deedle Formatter

In order to better inspect the content of the data frames and serieses (roughly translated: columns in a data frame) we need to format them correctly. The referenced extension combines the implementations for [a similar
Formatter using Microsoft's DataFrama](https://github.com/dotnet/interactive/blob/main/samples/notebooks/fsharp/Samples/HousingML.ipynb) and [IFSharp's Deedle formatter](https://github.com/mndrake/IfSharpLab/blob/master/src/DeedleFormat.fs). For instructions how to use the extension see [its readme](https://github.com/WalternativE/Deedle.DotNet.Interactive.Extension).

In [1]:
#!fsharp
#r "nuget: Deedle.DotNet.Interactive.Extension,0.1.0-alpha1"

Installed package Deedle.DotNet.Interactive.Extension version 0.1.0-alpha1

Loaded Deedle.DotNet.Interactive.Extension.DeedleFormatterExtension

Added DeedleFormatterExtension including formatters for Frame and Series

In [1]:
#!fsharp
open Deedle
open XPlot.Plotly
open System

In [1]:
#!fsharp
let flights = Frame.ReadCsv "/home/gregor/source/repos/FSharpForDataScience/datasets/nycflights13/flights.csv"

In [1]:
#!fsharp
flights

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,...
,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.String,System.Int32,System.String,System.String,System.String,System.Int32,...
0,2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,...
1,2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,...
2,2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,...
3,2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,...
4,2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,...
5,2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,...
6,2013,1,1,555,600,-5,913,854,19,B6,507,N516JB,EWR,FLL,158,...
7,2013,1,1,557,600,-3,709,723,-14,EV,5708,N829AS,LGA,IAD,53,...
8,2013,1,1,557,600,-3,838,846,-8,B6,79,N593JB,JFK,MCO,140,...


In [1]:
#!fsharp
let alaskaFlights =
    flights
    |> Frame.filterRowsBy "carrier" "AS"

alaskaFlights

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,...
,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.Int32,System.String,System.Int32,System.String,System.String,System.String,System.Int32,...
78,2013,1,1,724,725,-1,1020,1030,-10,AS,11,N594AS,EWR,SEA,338,...
644,2013,1,1,1808,1815,-7,2111,2130,-19,AS,7,N553AS,EWR,SEA,336,...
949,2013,1,2,722,725,-3,949,1030,-41,AS,11,N592AS,EWR,SEA,314,...
1573,2013,1,2,1818,1815,3,2131,2130,1,AS,7,N552AS,EWR,SEA,332,...
1894,2013,1,3,724,725,-1,1012,1030,-18,AS,11,N533AS,EWR,SEA,325,...
2489,2013,1,3,1817,1815,2,2121,2130,-9,AS,7,N552AS,EWR,SEA,327,...
2812,2013,1,4,725,725,0,1031,1030,1,AS,11,N579AS,EWR,SEA,345,...
3400,2013,1,4,1808,1815,-7,2101,2130,-29,AS,7,N553AS,EWR,SEA,338,...
3696,2013,1,5,725,725,0,1011,1030,-19,AS,11,N546AS,EWR,SEA,330,...


In [1]:
#!fsharp
let arrivalDelayOnDepDelayTrace =
    let depDelays = alaskaFlights?dep_delay |> Series.fillMissingWith 0. |> Series.values
    let arrDelays = alaskaFlights?arr_delay |> Series.fillMissingWith 0. |> Series.values
    Scatter(
        x = depDelays,
        y = arrDelays,
        mode = "markers",
        marker = Marker(
            opacity = 0.2
        )
    )

[ arrivalDelayOnDepDelayTrace ]
|> Chart.Plot

In [1]:
#!fsharp
let inline jitter (rnd: Random) (width: ^W) (series: Series<_, ^T>) =
    series
    |> Series.map (fun _ v ->
        let half = (float width) / 2.
        (rnd.NextDouble() * (float width) - half) + (double v))

In [1]:
#!fsharp
let arrivalDelayOnDepDelayTrace =
    let jitter' = jitter (Random(Seed = 1)) 10.
    let depDelays =
        alaskaFlights?dep_delay
        |> Series.fillMissingWith 0.
        |> jitter'
        |> Series.values
    let arrDelays =
        alaskaFlights?arr_delay
        |> Series.fillMissingWith 0.
        |> jitter'
        |> Series.values
    Scatter(
        x = depDelays,
        y = arrDelays,
        mode = "markers"
    )

[ arrivalDelayOnDepDelayTrace ]
|> Chart.Plot

In [1]:
#!fsharp
let weather = Frame.ReadCsv "/home/gregor/source/repos/FSharpForDataScience/datasets/nycflights13/weather.csv"

In [1]:
#!fsharp
weather

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
,System.String,System.Int32,System.Int32,System.Int32,System.Int32,System.Decimal,System.Decimal,System.Decimal,System.Double,System.Decimal,System.Double,System.Int32,System.Double,System.Int32,System.DateTime
0,EWR,2013,1,1,1,39.02,26.06,59.37,270,10.35702,N/A,0,1012,10,01/01/2013 01:00:00
1,EWR,2013,1,1,2,39.02,26.96,61.63,250,8.05546,N/A,0,1012.3,10,01/01/2013 02:00:00
2,EWR,2013,1,1,3,39.02,28.04,64.43,240,11.5078,N/A,0,1012.5,10,01/01/2013 03:00:00
3,EWR,2013,1,1,4,39.92,28.04,62.21,250,12.65858,N/A,0,1012.2,10,01/01/2013 04:00:00
4,EWR,2013,1,1,5,39.02,28.04,64.43,260,12.65858,N/A,0,1011.9,10,01/01/2013 05:00:00
5,EWR,2013,1,1,6,37.94,28.04,67.21,240,11.5078,N/A,0,1012.4,10,01/01/2013 06:00:00
6,EWR,2013,1,1,7,39.02,28.04,64.43,240,14.96014,N/A,0,1012.2,10,01/01/2013 07:00:00
7,EWR,2013,1,1,8,39.92,28.04,62.21,250,10.35702,N/A,0,1012.2,10,01/01/2013 08:00:00
8,EWR,2013,1,1,9,39.92,28.04,62.21,260,14.96014,N/A,0,1012.7,10,01/01/2013 09:00:00


In [1]:
#!fsharp
let earlyJanuaryWeather =
    weather
    |> Frame.filterRowsBy "origin" "EWR"
    |> Frame.filterRowValues (fun row -> row.GetAs<int>("month") = 1)
    |> Frame.filterRowValues (fun row -> row.GetAs<int>("day") <= 15)

earlyJanuaryWeather

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
,System.String,System.Int32,System.Int32,System.Int32,System.Int32,System.Decimal,System.Decimal,System.Decimal,System.Double,System.Decimal,System.Double,System.Int32,System.Double,System.Int32,System.DateTime
0,EWR,2013,1,1,1,39.02,26.06,59.37,270,10.35702,N/A,0,1012,10,01/01/2013 01:00:00
1,EWR,2013,1,1,2,39.02,26.96,61.63,250,8.05546,N/A,0,1012.3,10,01/01/2013 02:00:00
2,EWR,2013,1,1,3,39.02,28.04,64.43,240,11.5078,N/A,0,1012.5,10,01/01/2013 03:00:00
3,EWR,2013,1,1,4,39.92,28.04,62.21,250,12.65858,N/A,0,1012.2,10,01/01/2013 04:00:00
4,EWR,2013,1,1,5,39.02,28.04,64.43,260,12.65858,N/A,0,1011.9,10,01/01/2013 05:00:00
5,EWR,2013,1,1,6,37.94,28.04,67.21,240,11.5078,N/A,0,1012.4,10,01/01/2013 06:00:00
6,EWR,2013,1,1,7,39.02,28.04,64.43,240,14.96014,N/A,0,1012.2,10,01/01/2013 07:00:00
7,EWR,2013,1,1,8,39.92,28.04,62.21,250,10.35702,N/A,0,1012.2,10,01/01/2013 08:00:00
8,EWR,2013,1,1,9,39.92,28.04,62.21,260,14.96014,N/A,0,1012.7,10,01/01/2013 09:00:00


In [1]:
#!fsharp
let earlyJanuaryWeatherLine =
    Scatter(
        x = (earlyJanuaryWeather.GetColumn<DateTime>("time_hour") |> Series.values),
        y = (earlyJanuaryWeather?temp |> Series.values),
        mode = "line"
    )

[ earlyJanuaryWeatherLine ]
|> Chart.Plot

In [1]:
#!fsharp
let histTrace (binCount: int) =
    let min = weather?temp |> Stats.min
    let max = weather?temp |> Stats.max
    let binSize =
        float (max - min) / (float binCount)
        |> Math.Round
        |> int
    Histogram(
        x = (weather?temp |> Series.values),
        autobinx = false,
        xbins = Xbins(
            start = min,
            ``end`` = max,
            size = binSize
        )
    )

let layout =
    Layout(
        bargap = 0.1
    )

[ histTrace 30 ]
|> Chart.Plot
|> Chart.WithLayout layout

In [1]:
#!fsharp
[ histTrace 10 ]
|> Chart.Plot
|> Chart.WithLayout layout